In [1]:
import mlflow
import pandas as pd

In [2]:
# No incluir columnas
exclude = ['Posted On', 'Area Locality','Floor']
data = pd.read_csv('raw.csv')

In [6]:
# mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_tracking_uri('http://ec2-54-165-160-184.compute-1.amazonaws.com:5000')

In [7]:
client = mlflow.tracking.MlflowClient()

In [15]:
experiments = (client.search_experiments())
experiment  = list(filter(lambda k: k.name == 'model_rent_lgbm', experiments))[0]
experiment_id = experiment.experiment_id

In [17]:
from mlflow.entities import ViewType
runs = mlflow.MlflowClient().search_runs(
    experiment_ids=experiment_id,
    filter_string='',
    max_results=3,
    order_by=['metrics.MAE ASC'],
    run_view_type=ViewType.ACTIVE_ONLY
)

In [19]:
run = runs[0]

In [21]:
run_id = run.data.tags['Run ID']

In [23]:
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.

loaded_model.predict(data.drop(columns=exclude+['Rent']))

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


array([12677.37053666, 11971.8159973 ,  8980.08247125, ...,
       38607.39258275, 28713.4683281 , 16998.8463084 ])

In [25]:
mlflow.register_model(f'runs:/{run_id}/model', 'rent_model')

Registered model 'rent_model' already exists. Creating a new version of this model...
2023/11/02 20:43:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: rent_model, version 2
Created version '2' of model 'rent_model'.


<ModelVersion: aliases=[], creation_timestamp=1698975808151, current_stage='None', description='', last_updated_timestamp=1698975808151, name='rent_model', run_id='2ca1701d1142453f97a6dd7d88669748', run_link='', source='s3://pea-dmc-bucket/879546609870824623/2ca1701d1142453f97a6dd7d88669748/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [27]:
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name='rent_model',
    version='1',
    stage='Production'
)

<ModelVersion: aliases=[], creation_timestamp=1698975771841, current_stage='Production', description='', last_updated_timestamp=1698975924210, name='rent_model', run_id='2ca1701d1142453f97a6dd7d88669748', run_link='', source='s3://pea-dmc-bucket/879546609870824623/2ca1701d1142453f97a6dd7d88669748/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [28]:
logged_model = 'models:/rent_model/1'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.

loaded_model.predict(data.drop(columns=exclude+['Rent']))

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


array([12677.37053666, 11971.8159973 ,  8980.08247125, ...,
       38607.39258275, 28713.4683281 , 16998.8463084 ])

In [29]:
logged_model = 'models:/rent_model/Production'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.

loaded_model.predict(data.drop(columns=exclude+['Rent']))

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


array([12677.37053666, 11971.8159973 ,  8980.08247125, ...,
       38607.39258275, 28713.4683281 , 16998.8463084 ])